# Filtering by Location

## Setup

In [ ]:
from collections import Counter
from tqdm import tqdm 
from colorama import Style,Fore
import os
from dotenv import load_dotenv
import json
import time
load_dotenv()

In [ ]:
def openJson(path):
    
    # Creates the file if not existing
    if not os.path.exists(path):
        with open(path, "w", encoding="utf-8") as file:
            json.dump([], file)
            
    # Open it otherwise
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

def saveJson(path,data):
    with open(path, "w", encoding="utf-8") as f:
       json.dump(data, f, ensure_ascii=False, indent=2)
       #print(Style.BRIGHT+Fore.GREEN+'\n json saved'+Style.RESET_ALL)

## Run All

In [ ]:
channelsR1 = openJson("../videos/jsons/channelsR1.json")
len(channelsR1)

In [ ]:
channelsF1 = []
for channel in channelsR1:
    if channel['localisation']=='FR':
        channelsF1.append(channel)

saveJson("./jsons/channelsF1.json",channelsF1)
print(len(channelsF1))

# Filtering of TV Channels

In [ ]:
chainesTv = [
    "France 2", "France 3", "France 4","France 5","Franceinfo",
    "BFMTV", "C8", "CStar", "Gulli", "Cnews",
    "Canal+", "Planète+", "LCI", "Paris première",
    "6ter", "Arte", "M6", "W9",
    "TFX", "TMC", "NRJ12", "TF1","La Chaîne parlementaire",
    "Chérie 25", "RMC"
]
chainesTv = [nomTV.lower().replace(" ", "") for nomTV in chainesTv]

print(len(chainesTv))
print(chainesTv)

In [ ]:
channelsF2 = []
for channel in channelsF1:
    isTV = False
    for nameTV in chainesTv:
        if nameTV in channel['nom_chaine'].lower().replace(" ", ""):
            isTV = True
    if isTV == False:
        channelsF2.append(channel)

saveJson("./jsons/channelsF2.json",channelsF2)
len(channelsF2)

# Filtering by Relevance

## Short Bio problem resolution

- We aim to use the Bios of channels in this step, but some channels have empty or very small Bios so  we need first to fix this issue.
- An alternative context that represents a channel can be the metadata (title, description, tags) of 3 last videos shared by the channel.


In [ ]:
from scrapetube import get_channel
from googleapiclient.discovery import build

def getContext(channel_id):
    api_key = os.getenv("YOUTUBE_API_KEY")
    youtube = build('youtube', 'v3', developerKey=api_key)

    video_ids = []
    for video in get_channel(channel_id, limit=3, sort_by='newest', content_type='videos'):
        video_ids.append(video['videoId'])

    if not video_ids:
        return "No videos found."

    request = youtube.videos().list(part='snippet', id=','.join(video_ids))
    response = request.execute()

    combined = ""
    for item in response['items']:
        title = item['snippet'].get('title', 'No title')
        description = item['snippet'].get('description', 'No description')
        tags = ', '.join(item['snippet'].get('tags', [])) if 'tags' in item['snippet'] else 'No tags'

        combined += f"Title: {title}\nDescription: {description}\nTags: {tags}\n\n"

    return combined


In [ ]:
saveCount = 0
for item in tqdm(channelsF2):
    bio = item.get('bio', '').strip()
    nb_mots = len(bio.split())
    
    if nb_mots <= 100 :
        context = getContext(item['id_chaine'])
        item['context'] = context
        
    saveCount+=1
    if saveCount >= 100:
        saveJson("./jsons/channelsF2.json",channelsF2)
        saveCount = 0
   
saveJson("./jsons/channelsF2.json",channelsF2)

## LLM

- An LLM gives a decision about the relevance of a channel based on it's bio or context(3 last videos metadata)

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
gemini_flash = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
sysprompt = """
# Définition de l’autosuffisance

L'autosuffisance est une démarche visant à acquérir la capacité de subvenir par soi-même à ses besoins fondamentaux,  
en priorité alimentaires, par l'autoconsommation – c'est-à-dire produire, récolter et conserver un maximum de sa propre nourriture,  
souvent en privilégiant le bio, le local et le saisonnier.

Elle dépasse la simple autonomie matérielle : c’est un engagement volontaire pour réduire sa dépendance au système économique et social extérieur.  
Cela implique des choix concrets : trouver un lieu adapté, l’aménager (par exemple en permaculture), repenser sa manière de consommer et de gérer son temps, pour construire une vie plus autonome.

---

# Votre mission

Décider si une chaîne est directement liée à la thématique de l’autosuffisance, en se basant sur un des deux contextes suivants :
- **context** = nom de la chaîne + bio
- OU  
- **context** = nom de la chaîne + bio + titres, descriptions et tags des trois dernières vidéos (si la bio est courte ou vide)

---

# Critères de décision

## Répondre **"oui"** uniquement si la chaîne est portée par :

- Des personnes ou des groupes qui cherchent à devenir autonomes (énergie, alimentation, habitat, etc.),  
  soit à titre **individuel** (seuls ou en famille),  soit de manière **auto-organisée** (collectifs informels, association ...).

Et uniquement si le contenu est **centré sur l’autosuffisance** :  

potager, autonomie alimentaire, autonomie énergétique, conservation, vie autonome, permaculture, etc.

## La présence de mots-clés comme :  

**autosuffisance**, **autosuffisant**, **autonomie**, **vie autonome**, **autonomie alimentaire**, **autonomie énergétique**  

est un bon indicateur.

## Exclure systématiquement :

- Les chaînes d'entreprises, d’institutions ou de structures commerciales (À but lucratif).
- Les chaînes d’agriculture générale, de jardinage standard ou de permaculture académique, **si elles ne sont pas clairement orientées vers une démarche d’autosuffisance personnelle ou collective non institutionnelle**.

---

# Format attendu

Répondre dans ce format JSON :

  "decision": "oui" ou "non",
  "justification": "Justifiez votre décision avec des arguments clairs, en vous basant uniquement sur les critères définis ci-dessus."

"""

userprompt = """
Le context ici
---
{context}

"""

prompt = ChatPromptTemplate([
    ("system", sysprompt),
    ("user", userprompt)
])

chain = prompt | gemini_flash


In [ ]:
#Test

llmInput = "\n                              Nom Chaine :\n                              ---\n                              L’Ôton’home\n                              \n                              Bio Chaine :\n                              ---\n                              Nous sommes un couple parent de 2 jeunes garçons et avons acheté à l’été 2020 une GRANDE GRANGE sur un peu plus de 3 hectares de terrain dans le Tarn. Notre objectif: transformer cette grange en HABITATION la plus AUTONOME possible, ce par nos propres moyens, et en faire notre petit paradis. Venez partager notre aventure et suivre ce projet un peu fou ici.\n\nLe but de notre chaîne: montrer toutes les étapes de la TRANSFORMATION de la GRANGE en HABITATION, échanger sur l’AUTONOMIE, partager nos DIY en matière de PRODUITS MÉNAGERS et COSMÉTIQUES, toujours dans une optique d’AUTONOMIE.\n\nPrécision importante: nous ne sommes PAS des professionnels du bâtiment donc nos vidéos n'ont pas valeur de tutoriels mais de simples partages d'expérience.\n\nN’hésitez pas à laisser vos commentaires et à vous abonner à la chaîne. Vous pouvez également nous retrouver sur Facebook et Tipeee. En espérant pouvoir vous inspirer.\n\nDes bises,\n\nAurélie & David \n                            ",
input = {"context":llmInput}
print(input)
print(chain.invoke(input))

## Run All

In [ ]:
channelsF2 = openJson("./jsons/channelsF2.json")
len(channelsF2)

In [ ]:
# Prepare llmInput

for channel in channelsF2:
    llmInput = ""
    if "context" in channel:
        llmInput = f"""
                    Nom Chaine :
                    ---
                    {channel['nom_chaine']}
                    
                    Bio Chaine :
                    ---
                    {channel['bio']}
                    
                    contexte d’après les vidéos :
                    ---
                    {channel['context']}   
                """          
    else :
        llmInput = f"""
                    Nom Chaine :
                    ---
                    {channel['nom_chaine']}
                    
                    Bio Chaine :
                    ---
                    {channel['bio']} 
                """
    channel['llmInput'] = llmInput   
    
saveJson("./jsons/channelsF2.json",channelsF2) 

In [ ]:
def cleanAnswer(answer):
    answer = answer.strip("`")   
    if answer.startswith("json"):
        answer = answer[4:].strip() 
    return  answer

In [ ]:
saveCount = 0
apiCount = 0

for item in tqdm(channelsF2):
    answer = cleanAnswer(chain.invoke({"context":item["llmInput"]}))
    answer = json.loads(answer)
    #print(video['id_video'],'/n',answer)
    item.update(answer)
    
    saveCount+=1
    apiCount+=1
    
    # API Delay of 1 min 
    if apiCount == 13:
        print("sleep for 60 s")
        time.sleep(60)
        apiCount=0
        
    # Saving Safety
    if saveCount >= 100:
        saveJson("./jsons/channelsF2.json",channelsF2)
        saveCount = 0
   
saveJson("./jsons/channelsF2.json",channelsF2)

In [ ]:
channelsF3 = []
channelsF3Non = []

for channel in channelsF2:
    if channel['decision']=='oui':
      channelsF3.append(channel) 
    else :
        channelsF3Non.append(channel) 

saveJson("./jsons/channelsF3.json",channelsF3)
saveJson("./jsons/channelsF3Non.json",channelsF3Non)

print(len(channelsF3))
print(len(channelsF3Non))